In [1]:
!pip install rdkit


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
import typing
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem, Descriptors
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import FunctionTransformer
import warnings
import random

warnings.filterwarnings('ignore')

In [55]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [2]:
data = pd.read_excel('data/All-1614.xlsx')
data.head()

,Title,"IC50, mmg/ml","CC50-MDCK, mmg/ml",SI,Molecular weight,Hydrogen bond acceptors,Hydrogen bond donors,Polar SA,SMILES,Pictures
0,1007-Ya-213,2.7,500.0,185.185185,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,50.0
1,1007-Ya-213,0.7,447.0,638.571429,195.307,2,1,32.59,OCC\N=C(\[C@]12C)C[C@@H](C1(C)C)CC2,51.0
2,1008-Ya-187,9.9,144.0,14.545455,250.431,1,0,15.60,CCN(CC)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,52.0
3,1009-As-106,8.3,500.0,60.240964,222.377,1,0,15.60,CN(C)CC\N=C(\[C@@]12C)C[C@H](C1(C)C)CC2,53.0
4,1010-Ya-208,39.4,143.0,3.629442,239.361,2,0,29.54,CN(C)CC(=O)O[C@H]1C[C@H](CC2)C(C)(C)[C@@]12C,54.0


In [3]:
data.rename(columns={'IC50, mmg/ml': 'IC50', 'CC50-MDCK, mmg/ml': 'CC50'}, inplace=True)

In [4]:
def mol_dsc_calc(mols):
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in mols)


# список конституционных и физико-химических дескрипторов из библиотеки RDKit
descriptors = {"HeavyAtomCount": Descriptors.HeavyAtomCount,
               "NHOHCount": Descriptors.NHOHCount,
               "NOCount": Descriptors.NOCount,
               "NumHAcceptors": Descriptors.NumHAcceptors,
               "NumHDonors": Descriptors.NumHDonors,
               "NumHeteroatoms": Descriptors.NumHeteroatoms,
               "NumRotatableBonds": Descriptors.NumRotatableBonds,
               "NumValenceElectrons": Descriptors.NumValenceElectrons,
               "NumAromaticRings": Descriptors.NumAromaticRings,
               "NumAliphaticHeterocycles": Descriptors.NumAliphaticHeterocycles,
               "RingCount": Descriptors.RingCount,
               "MW": Descriptors.MolWt,
               "LogP": Descriptors.MolLogP,
               "MR": Descriptors.MolMR,
               "TPSA": Descriptors.TPSA}

# sklearn трансформер для использования в конвейерном моделировании
descriptors_transformer = FunctionTransformer(mol_dsc_calc)

In [5]:
def rdkit_fp(smiles_column: pd.Series, radius=3, nBits=2048, useChirality=False):
    # morganFP_rdkit
    def desc_gen(mol):
        mol = Chem.MolFromSmiles(mol)
        bit_vec = np.zeros((1,), np.int16)
        DataStructs.ConvertToNumpyArray(
            AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=nBits, useChirality=useChirality), bit_vec)
        return bit_vec

    return pd.DataFrame.from_records(smiles_column.apply(func=desc_gen), columns=[f'bit_id_{i}' for i in range(nBits)])


def rdkit_2d(smiles_column: pd.Series):
    # 2d_rdkit
    descriptors = {i[0]: i[1] for i in Descriptors._descList}
    return pd.DataFrame({k: f(Chem.MolFromSmiles(m)) for k, f in descriptors.items()} for m in smiles_column)

In [6]:
from dataclasses import dataclass


@dataclass
class SMILESDescriptor:
    descriptor: typing.Callable
    title: typing.Optional[str] = None

In [26]:
class SMILESMolDataset(Dataset):
    def __init__(
            self,
            *,
            data: pd.DataFrame,
            x_columns: typing.List[str],
            y_columns: typing.List[str],
            x_transform: typing.Optional[typing.Callable] = None,
            y_transform: typing.Optional[typing.Callable] = None,
            smiles_descriptors: typing.Optional[typing.Iterable[SMILESDescriptor]] = None
    ):
        self.data = data
        self.x_columns = x_columns
        self.y_columns = y_columns
        self.x_transform = x_transform
        self.y_transform = y_transform
        self.smiles_descriptors = smiles_descriptors

        self._descript_smiles()
        self._separate_data()

    def _descript_smiles(self):
        if self.smiles_descriptors:
            for d in self.smiles_descriptors:
                d_data = d.descriptor(self.data['SMILES'])
                self.data = self.data.join(d_data, lsuffix=d.title or '')

    def _separate_data(self):
        x = self.data[self.x_columns]
        if self.x_transform:
            for column in self.x_columns:
                x[column] = x[column].apply(self.x_transform)
        self.x = x

        y = self.data[self.y_columns]
        if self.y_transform:
            for column in self.y_columns:
                y[column] = y[column].apply(self.y_transform)
        self.y = y

    def columns(self):
        return self.data.columns

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return self.x.loc[idx].values, self.y.loc[idx].values

In [62]:
train_data = data.sample(frac=0.8, random_state=seed)
test_data = data.drop(train_data.index)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [63]:
dataset_args = {
    'x_columns': ['IC50', 'SI', 'Molecular weight'],
    'y_columns': ['CC50'],
    'x_transform': None,
    'y_transform': lambda y: int(y >= 4),
    'smiles_descriptors': [
        SMILESDescriptor(descriptors_transformer.transform),
        SMILESDescriptor(rdkit_fp, 'rdkit_fp'),
        SMILESDescriptor(rdkit_2d, 'rdkit_2d')
    ]
}

train_dataset = SMILESMolDataset(data=train_data, **dataset_args)
test_dataset = SMILESMolDataset(data=test_data, **dataset_args)

In [64]:
train_dataset[1]

(array([100.   ,   2.8  , 167.253]), array([1], dtype=int64))

In [68]:
class TrainingModel:
    def __init__(
            self,
            *,
            model: typing.Any,
            fit_method: str = 'fit',
            predict_method: str = 'predict',
            **kwargs
    ):
        self.model = model(**kwargs)
        self._fit_method = fit_method
        self._predict_method = predict_method

    def fit(self, x_train: pd.DataFrame, y_train: pd.DataFrame, **kwargs):
        getattr(self.model, self._fit_method)(x_train, y_train, **kwargs)

    def predict(self, x_test: pd.DataFrame, **kwargs) -> typing.Union[np.ndarray]:
        return getattr(self.model, self._predict_method)(x_test, **kwargs)

In [43]:
!pip install catboost

   ---------------------------------------- 0.0/101.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.0 MB 1.1 MB/s eta 0:01:33
   ---------------------------------------- 0.2/101.0 MB 2.3 MB/s eta 0:00:45
   ---------------------------------------- 0.5/101.0 MB 3.7 MB/s eta 0:00:28
    --------------------------------------- 1.3/101.0 MB 6.7 MB/s eta 0:00:15
   - -------------------------------------- 2.6/101.0 MB 11.2 MB/s eta 0:00:09
   - -------------------------------------- 3.9/101.0 MB 15.5 MB/s eta 0:00:07
   - -------------------------------------- 3.9/101.0 MB 15.5 MB/s eta 0:00:07
   -- ------------------------------------- 6.6/101.0 MB 18.4 MB/s eta 0:00:06
   --- ------------------------------------ 8.7/101.0 MB 21.4 MB/s eta 0:00:05
   --- ------------------------------------ 9.2/101.0 MB 21.7 MB/s eta 0:00:05
   ---- ----------------------------------- 11.4/101.0 MB 31.2 MB/s eta 0:00:03
   ----- ---------------------------------- 14.5/101.0 MB 54.4 

In [69]:
from catboost import CatBoostClassifier

training_model = TrainingModel(
    model=CatBoostClassifier,
    iterations=100
)

training_model.fit(train_dataset.x, train_dataset.y)

Learning rate set to 0.094876
0:	learn: 0.6267560	total: 1.26ms	remaining: 125ms
1:	learn: 0.5698490	total: 2.19ms	remaining: 107ms
2:	learn: 0.5187655	total: 2.98ms	remaining: 96.4ms
3:	learn: 0.4752630	total: 3.78ms	remaining: 90.8ms
4:	learn: 0.4378315	total: 4.56ms	remaining: 86.6ms
5:	learn: 0.4011325	total: 5.36ms	remaining: 84ms
6:	learn: 0.3743672	total: 6.13ms	remaining: 81.4ms
7:	learn: 0.3477135	total: 6.88ms	remaining: 79.1ms
8:	learn: 0.3234805	total: 7.7ms	remaining: 77.9ms
9:	learn: 0.3049522	total: 8.49ms	remaining: 76.4ms
10:	learn: 0.2834217	total: 9.28ms	remaining: 75.1ms
11:	learn: 0.2623452	total: 10.1ms	remaining: 73.9ms
12:	learn: 0.2444954	total: 10.9ms	remaining: 73ms
13:	learn: 0.2299818	total: 11.7ms	remaining: 71.9ms
14:	learn: 0.2144001	total: 12.5ms	remaining: 71ms
15:	learn: 0.2043646	total: 13.3ms	remaining: 69.9ms
16:	learn: 0.1925611	total: 14.1ms	remaining: 69ms
17:	learn: 0.1806978	total: 15ms	remaining: 68.1ms
18:	learn: 0.1734262	total: 15.3ms	rema

In [70]:
y_pred = training_model.predict(test_dataset.x)

In [72]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,